In [1]:
#getting and working with data
import pandas as pd
import numpy as np
import re
import os
import datetime as dt
import string

#visualizing results
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set_context("poster")
sns.set_style("ticks")
#import yellowbrick as yb

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

import warnings; warnings.simplefilter('ignore')
np.set_printoptions(suppress=True)

C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\__init__.py:29: UserWarning: loaded more than 1 DLL from .libs:
C:\ProgramData\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.CSRRD7HKRKC3T3YXA7VY7TAZGLSWDKW6.gfortran-win_amd64.dll
C:\ProgramData\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
  stacklevel=1)


In [56]:
path_data = 'C:/Users/Schindler/Documents/Schindler_Lab/Data/Clinical projects/PeskindTBI/freeze_191003/TBIFreeze_20191003.xlsx'

#original file has multiple sheets (for different data set types (e.g. single visit, imaging etc)
#only interested in deploayed controls ('C') and mTBI ('T')
#each participant can have multiple visits - we are currently intersted in data only from the first visit

#we want to create one data frame with one row per participant corresponding to their first study visit and data across columns
first_visit_data = pd.DataFrame()

whole_file = pd.ExcelFile(path_data)
print(whole_file.sheet_names, '\n')

#each data set uses as different column name for visit date, use a dictionary to access corresponding date column name
sheet_dic = {'Single_Record_Measures': 'ScreenDate', 
             'Multi_Record_Measures': 'MeasureDate', 
             'TBI_Symptom_Q': 'NSIFormDate', 
             'DTI_Kleinhans_Lab': 'ScreenDate', 
             'FDG-PET': 'ScreenDate', 
             'Clinical_Labs': 'CLabDate'}

for sheet in whole_file.sheet_names:
    print('Sheet being processed:\n', sheet)
    
    #create intermediate dataframe
    data_int = pd.DataFrame(data = pd.read_excel(whole_file, sheet))

    #select only TBIID C and T (control and TBI)
    data_int = data_int[data_int['TBIID'].str.match(r'[CT]\d\d')]
    print('Data shape only deployed controls and mTBI groups:\n', data_int.shape)

    # create new data frame containing only the first visit record (based on 'ScreenDate' column)
    #add new column with group ID (C = deployed controls, T = blast mTBI)
    first_visit_data_int = pd.DataFrame()

    participants = data_int['TBIID'].unique()
    print('Number of participants:\n', len(participants))

    for part in participants:
        dates = data_int.loc[data_int['TBIID'] == part, sheet_dic[sheet]].values
        min_date = dates.min()
    
        first_date = data_int[(data_int['TBIID'] == part) & (data_int[sheet_dic[sheet]] == min_date)]
        first_visit_data_int = first_visit_data_int.append(first_date)
    
    #reset indexes and add to final dataframe
    first_visit_data_int = first_visit_data_int.reset_index(drop=True)
    #clean up missing values
    first_visit_data_int = first_visit_data_int.replace({-999.0: np.nan, 'None': np.nan})
    
    if first_visit_data.shape[0] < 1:
        first_visit_data = first_visit_data_int
    else:
        first_visit_data = pd.merge(first_visit_data, first_visit_data_int, how='left', on=['TBIID'])
    print('First visit data shape for current sheet:\n', first_visit_data.shape, '\n')

print('Final shape of first visit data:\n', first_visit_data.shape)
first_visit_data.head()

['Single_Record_Measures', 'Multi_Record_Measures', 'TBI_Symptom_Q', 'DTI_Kleinhans_Lab', 'FDG-PET', 'Clinical_Labs'] 

Sheet being processed:
 Single_Record_Measures
Data shape only deployed controls and mTBI groups:
 (215, 835)
Number of participants:
 144
First visit data shape for current sheet:
 (145, 835) 

Sheet being processed:
 Multi_Record_Measures
Data shape only deployed controls and mTBI groups:
 (271, 84)
Number of participants:
 144
First visit data shape for current sheet:
 (145, 918) 

Sheet being processed:
 TBI_Symptom_Q
Data shape only deployed controls and mTBI groups:
 (265, 42)
Number of participants:
 144
First visit data shape for current sheet:
 (145, 959) 

Sheet being processed:
 DTI_Kleinhans_Lab
Data shape only deployed controls and mTBI groups:
 (212, 196)
Number of participants:
 144
First visit data shape for current sheet:
 (145, 1154) 

Sheet being processed:
 FDG-PET
Data shape only deployed controls and mTBI groups:
 (212, 126)
Number of participant

,EntityID,DOB,GType,Race,Hispanic,Handedness,Status_x,StatusChangedDate,KFPSubject,TBIID,ScreenDate_x,VisitSeq_x,ScreenAge,Education,branchAF,branchArm,branchCG,branchMar,branchNav,branchNG,branchOth,MOS,MOS2,MOS3,MOS4,MOS5,Marital,servconn,cestotal,ExpPB,ExpMark1,ExpAntiM,ExpStim,ExpOthrS,ExpDEET,ExpTick,ExpPCollr,ExpPStrp,ExpPEnvi,ExpToxic,ExpPaint,ExpXsVib,ExpHStrk,ExpRadar,ExpIonRa,ExpYCake,ExpVhicl,ExpUrRnd,ExpDtOrd,ExpGasM,ExpMOPP,ExpRadBg,ExpAN,ExpNG,ExpTNT,ExpPETN,ExpRDX,ExpNC,ExpANFO,ExpCompB,ExpOctol,ExpPntlt,ExpDynmt,ExpOthrX,TBIMilitaryTBIID,StartDate1,EndDate1,StartDate2,EndDate2,StartDate3,EndDate3,StartDate4,EndDate4,StartDate5,EndDate5,CognitiveTBIID,PsyEduc,MMSEORDA,MMSEORLO,MMSE,DIGIF3,DIGIFLEN3,DIGIB3,DIGIBLEN3,DIGITOT3,DIGIPER3,ANIMALS,TRAILA,TRAILB,LETTERS,LETTERF,LETTERA,AnimalsT,TrailAT,TrailBT,LetFAST,wtarfsiq,wtarviq,tommtot1,tommtot2,tommret,CVTr1FR,CVTr1FRT,CVT15FR,CVT15FRT,CVShDFR,CVShDFRT,CVShDCR,CVShDCRT,CVLoDFR,CVLoDFRT,CVLoDCR,CVLoDCRT,CVYNRH,CVYNRHT,CVYNFP,CVYNFPT,CVFCRRaw,CVDeInt,BVTr1,BVTr1T,BVTotRe,BVTotReT,BVLearn,BVLearnT,BVDelRe,BVDelReT,BVRecHit,BVRecFA,BVDeInt,FTDomH,FTDomHT,FTNDomH,FTNDomHT,W3LNS,W3LNSSS,GPgD,GPgDT,GPgND,GPgNDT,CPTOm,CPTOmT,CPTCom,CPTComT,CPTHRT,CPTHRTT,CPTHSE,CPTHSET,CPTVar,CPTVarT,CPTd,CPTdT,CPTB,CPTBT,CPTRTBl,CPTRTBlT,CPTSEBl,CPTSEBlT,CPTRTIs,CPTRTIsT,CPTSEIs,CPTSEIsT,WCSTPrs,WCSTPrsT,WCSTCL,WCSTCLP,WCSTCLPT,WCSTCat,WCSTFail,TowerItemAd,TowerFrstMove,TowerMove,NumItemAdmin,TowerRuleVio,TowerItemCom,TowerAchieve,TowerAchieveS,TowerMeanFrstMoveR,TowerMeanFrstMoveRS,TowerTimePerMoveR,TowerTimePerMoveRS,TowerMoveAccR,TowerMoveAccRS,TowerRuleVioPerc,TEA6tptS,TEA7dtdS,TEA8LotS,SDW90Raw,SDW90TS,SDO90Raw,SDO90TS,MTRRRaw,MTRRRPer,MT2Raw,MT2Per,MT15Raw,MT15Per,MTTCRaw,MTTCPer,MTECRaw,MTECPer,MTVRRaw,MTVRPer,MTARRaw,MTARPer,MTPMTRaw,MTPMTPer,MTDPRaw,MTClck,MTPMT1EC,MTPMT2EC,MTPMT3EC,MTPMT4EC,MTPMT5EC,MTPMT6EC,MTPMT7EC,MTPMT8EC,CvmtHitR,CvmtHitT,CvmtFaR,CvmtFaT,CvmtDprR,CvmtDprT,CvmtTotR,CvmtTotT,CvmtDelR,CvmtDelT,CvmtVis,CvmtInt,DkefCnR,DkefCnSS,DkefWrR,DkefWrSS,DkefInR,DkefInSS,DkefSwR,DkefSwSS,DkefCnEr,DkefWrEr,DkefInEr,DkefSwEr,DkefCnUn,DkefWrUn,DkefInUn,DkefSwUn,NDCompSc,NDRdRate,ACTFrmDate,ACT0,ACT9,...,DTI_MD_sup_long_fascic_R,DTI_MD_sup_long_fascic_L,DTI_MD_sup_frontocc_fascic_R,DTI_MD_sup_frontocc_fascic_L,DTI_MD_unc_fascic_R,DTI_MD_unc_fascic_L,DTI_MD_tapetum_R,DTI_MD_tapetum_L,subjects_,DTI_AD_mid_cerebll_pedunc,DTI_AD_pontine_xing_tract,DTI_AD_genu_cc,DTI_AD_body_cc,DTI_AD_splenium_cc,DTI_AD_fornix,DTI_AD_corticospinal_R,DTI_AD_corticospinal_L,DTI_AD_med_lemniscus_R,DTI_AD_med_lemniscus_L,DTI_AD_inf_cerebll_pedunc_R,DTI_AD_inf_cerebll_pedunc_L,DTI_AD_sup_cerebll_pedunc_R,DTI_AD_sup_cerebll_pedunc_L,DTI_AD_cerebr_pedunc_R,DTI_AD_cerebr_pedunc_L,DTI_AD_ant_int_capsule_R,DTI_AD_ant_int_capsule_L,DTI_AD_post_int_capsule_R,DTI_AD_post_int_capsule_L,DTI_AD_retrolenticular_int_capsule_R,DTI_AD_retrolenticular_int_capsule_L,DTI_AD_ant_corona_radiata_R,DTI_AD_ant_corona_radiata_L,DTI_AD_sup_corona_radiata_R,DTI_AD_sup_corona_radiata_L,DTI_AD_post_corona_radiata_R,DTI_AD_post_corona_radiata_L,DTI_AD_post_thal_rad_R,DTI_AD_post_thal_rad_L,DTI_AD_sag_stratum_R,DTI_AD_sag_stratum_L,DTI_AD_ext_capsule_R,DTI_AD_ext_capsule_L,DTI_AD_cingulum_cg_R,DTI_AD_cingulum_cg_L,DTI_AD_cingulum_hipp_R,DTI_AD_cingulum_hipp_L,DTI_AD_fornix_striaterm_R,DTI_AD_fornix_striaterm_L,DTI_AD_sup_long_fascic_R,DTI_AD_sup_long_fascic_L,DTI_AD_sup_frontocc_fascic_R,DTI_AD_sup_frontocc_fascic_L,DTI_AD_unc_fascic_R,DTI_AD_unc_fascic_L,DTI_AD_tapetum_R,DTI_AD_tapetum_L,DTI_RD_mid_cerebll_pedunc,DTI_RD_pontine_xing_tract,DTI_RD_genu_cc,DTI_RD_body_cc,DTI_RD_splenium_cc,DTI_RD_fornix,DTI_RD_corticospinal_R,DTI_RD_corticospinal_L,DTI_RD_med_lemniscus_R,DTI_RD_med_lemniscus_L,DTI_RD_inf_cerebll_pedunc_R,DTI_RD_inf_cerebll_pedunc_L,DTI_RD_sup_cerebll_pedunc_R,DTI_RD_sup_cerebll_pedunc_L,DTI_RD_cerebr_pedunc_R,DTI_RD_cerebr_pedunc_L,DTI_RD_ant_int_capsule_R,DTI_RD_ant_int_capsule_L,DTI_RD_post_int_capsule_R,DTI_RD_

In [59]:
first_visit_data.drop_duplicates(inplace=True)

(144,)

In [60]:
#create new column for sum of distance from blast (worst 5)
first_visit_data['QEDist_sum'] = first_visit_data.loc[:, 'QEDist1':'QEDist5'].sum(axis=1)
#create new column for mean of distance from blast (worst 5)
first_visit_data['QEDist_mean'] = first_visit_data.loc[:, 'QEDist1':'QEDist5'].mean(axis=1)
#create new column for min of distance from blast (worst 5)
first_visit_data['QEDist_min'] = first_visit_data.loc[:, 'QEDist1':'QEDist5'].min(axis=1)

In [ ]:
#now deal with path_RH
data_RH = pd.read_csv(path_RH)
data_RH = pd.DataFrame(data = data_RH)
print('Data shape all groups:\n', data_RH.shape, '\n')

#select only TBIID C and T (control and TBI)
data_RH = data_RH[data_RH['TBIID'].str.match(r'[CT]\d\d')]
print('Data shape only deployed controls and mTBI groups:\n', data_RH.shape, '\n')
print('Data types:\n', data_RH.info(), '\n')
data_RH.head(1)

In [ ]:
#change DA = 0 to np.nan, add ratio cals
data_RH['DA'] = data_RH['DA'].replace({0: np.nan})

data_RH['da_dopa_ratio'] = data_RH['DA'] / data_RH['DOPA']
data_RH['dopac_da_ratio'] = data_RH['DOPAC'] / data_RH['DA']
data_RH['ne_dopa_ratio'] = data_RH['NE'] / data_RH['DOPA']
data_RH['dhpg_ne_ratio'] = data_RH['DHPG'] / data_RH['NE']

In [ ]:
#further clean dataframe: only keep columns of interest and required metadata
first_visit_data_mult = first_visit_data_mult[['TBIID', 'Group', 'VisitSeq', 'MeasureDate', 'hrslp', 'PSQI1hr',
       'PSQI1min', 'PSQI2', 'PSQI3hr', 'PSQI3min', 'PSQI5a', 'PSQI5b',
       'PSQI5c', 'PSQI5d', 'PSQI5e', 'PSQI5f', 'PSQI5g', 'PSQI5h',
       'PSQI5i', 'PSQI5j', 'PSQI6', 'PSQI7', 'PSQI8', 'PSQI9', 'PSQIc1',
       'PSQIc2', 'PSQIc3', 'PSQIc4', 'PSQIc5', 'PSQIc6', 'PSQIc7',
       'PSQItot', 'PCL1', 'PCL2', 'PCL3', 'PCL4', 'PCL5', 'PCL6', 'PCL7',
       'PCL8', 'PCL9', 'PCL10', 'PCL11', 'PCL12', 'PCL13', 'PCL14',
       'PCL15', 'PCL16', 'PCL17', 'PCLTot', 'PHQ1', 'PHQ2', 'PHQ3',
       'PHQ4', 'PHQ5', 'PHQ6', 'PHQ7', 'PHQ8', 'PHQ9', 'PHQTot', 'auditc',
       'BNI1Im', 'BNI2Im', 'BNI3Im', 'BNI4Im', 'BNI5Im', 'BNI6Im',
       'BNI7Im', 'BNI8Im', 'BNI9Im', 'BNI10Im', 'BNI11', 'BNITotIm']]

first_visit_data_mult.columns = ['TBIID', 'Group', 'VisitSeq_mult', 'MeasureDate_mult', 'hrslp', 'PSQI1hr',
       'PSQI1min', 'PSQI2', 'PSQI3hr', 'PSQI3min', 'PSQI5a', 'PSQI5b',
       'PSQI5c', 'PSQI5d', 'PSQI5e', 'PSQI5f', 'PSQI5g', 'PSQI5h',
       'PSQI5i', 'PSQI5j', 'PSQI6', 'PSQI7', 'PSQI8', 'PSQI9', 'PSQIc1',
       'PSQIc2', 'PSQIc3', 'PSQIc4', 'PSQIc5', 'PSQIc6', 'PSQIc7',
       'PSQItot', 'PCL1', 'PCL2', 'PCL3', 'PCL4', 'PCL5', 'PCL6', 'PCL7',
       'PCL8', 'PCL9', 'PCL10', 'PCL11', 'PCL12', 'PCL13', 'PCL14',
       'PCL15', 'PCL16', 'PCL17', 'PCLTot', 'PHQ1', 'PHQ2', 'PHQ3',
       'PHQ4', 'PHQ5', 'PHQ6', 'PHQ7', 'PHQ8', 'PHQ9', 'PHQTot', 'AUDITtot',
       'BNI1Im', 'BNI2Im', 'BNI3Im', 'BNI4Im', 'BNI5Im', 'BNI6Im',
       'BNI7Im', 'BNI8Im', 'BNI9Im', 'BNI10Im', 'BNI11', 'BNITotIm']

In [ ]:
#further clean dataframe: only keep columns of interest and required metadata
first_visit_data_single = first_visit_data_single[['TBIID', 'VisitSeq', 
        'ScreenDate', 'GType', 'Race', 'Hispanic', 'Handedness',
       'ScreenAge', 'Education', 'cestotal', 'PsyEduc', 'QKOIorA', 'QKOExpMil', 'QKOAllMil',
       'QKOLife', 'QBlstExp', 'QBEACRM', 'MnthSncBlst', 'QEDist_sum', 'QEDist_mean', 'QEDist_min',
       'BISAtt', 'BISMtr', 'BISNonpl', 'BISTot', 'DvpVers', 'DvpHA',
       'DvpHAAct', 'DvpHASlp', 'DvpHAMd', 'DvpHAStr', 'DvpBP', 'DvpBPAct',
       'DvpBPslp', 'DvpBPMd', 'DvpBPStr', 'CAPSTotal']]

first_visit_data_single.columns = ['TBIID', 'VisitSeq_single', 
        'MeasureDate_single', 'GType', 'Race', 'Hispanic', 'Handedness',
       'ScreenAge', 'Education', 'cestotal', 'PsyEduc', 'QKOIorA', 'QKOExpMil', 'QKOAllMil',
       'QKOLife', 'QBlstExp', 'QBEACRM', 'MnthSncBlst', 'QEDist_sum', 'QEDist_mean', 'QEDist_min',
       'BISAtt', 'BISMtr', 'BISNonpl', 'BISTot', 'DvpVers', 'DvpHA',
       'DvpHAAct', 'DvpHASlp', 'DvpHAMd', 'DvpHAStr', 'DvpBP', 'DvpBPAct',
       'DvpBPslp', 'DvpBPMd', 'DvpBPStr', 'CAPSTotal']

In [ ]:
#further clean dataframe: only keep columns of interest and required metadata
first_visit_data_TBI = first_visit_data_TBI[['TBIID', 'VisitSeq', 'NSIFormDate',
                                             'tbiDizzy', 'tbiBalan', 'tbiCoord',
       'tbiHeada', 'tbiNaus', 'tbiVision', 'tbiLight', 'tbiHear',
       'tbiNoise', 'tbiTingl', 'tbiTstsml', 'tbiAppet', 'tbiConc',
       'tbiForget', 'tbiDecis', 'tbiSlow', 'tbiEnergy', 'tbiSleep',
       'tbiAnx', 'tbiSad', 'tbiIrrit', 'tbiOverw', 'tbiDisin', 'tbiWithd',
       'tbiRing', 'tbiMoods', 'tbiFight', 'tbiSpch', 'NSITot', 'TBITot']]

first_visit_data_TBI.columns = ['TBIID', 'VisitSeq_TBI', 'MeasureDate_TBI',
                                             'tbiDizzy', 'tbiBalan', 'tbiCoord',
       'tbiHeada', 'tbiNaus', 'tbiVision', 'tbiLight', 'tbiHear',
       'tbiNoise', 'tbiTingl', 'tbiTstsml', 'tbiAppet', 'tbiConc',
       'tbiForget', 'tbiDecis', 'tbiSlow', 'tbiEnergy', 'tbiSleep',
       'tbiAnx', 'tbiSad', 'tbiIrrit', 'tbiOverw', 'tbiDisin', 'tbiWithd',
       'tbiRing', 'tbiMoods', 'tbiFight', 'tbiSpch', 'NSITot', 'TBITot']

In [ ]:
#further clean dataframe: only keep columns of interest and required metadata
first_visit_data_PET = first_visit_data_PET[['TBIID', 'VisitSeq', 'ScreenDate', 'PETType', 'Frontal_Mid_l',
       'Frontal_Mid_r', 'Insula_l', 'Insula_r', 'Cingulum_Ant_l',
       'Cingulum_Ant_r', 'Amygdala_l', 'Amygdala_r', 'CaudateNucl_l',
       'CaudateNucl_r', 'Putamen_l', 'Putamen_r', 'Pallidum_l',
       'Pallidum_r', 'Medulla', 'Midbrain', 'Pons']]

first_visit_data_PET.columns = ['TBIID', 'VisitSeq_PET', 'MeasureDate_PET', 'PETType', 'Frontal_Mid_l',
       'Frontal_Mid_r', 'Insula_l', 'Insula_r', 'Cingulum_Ant_l',
       'Cingulum_Ant_r', 'Amygdala_l', 'Amygdala_r', 'CaudateNucl_l',
       'CaudateNucl_r', 'Putamen_l', 'Putamen_r', 'Pallidum_l',
       'Pallidum_r', 'Medulla', 'Midbrain', 'Pons']

In [ ]:
#further clean dataframe: only keep columns of interest and required metadata
first_visit_data_drug = first_visit_data_drug[['TBIID', 'VisitSeq', 'ScreenDate', 'PreCaff', 'PreETOH', 'PreNic', 'PreSleep', 
                                            'PreTHC']]

first_visit_data_drug.columns = ['TBIID', 'VisitSeq_drug', 'MeasureDate_drug', 'PreCaff', 'PreETOH', 'PreNic', 'PreSleep', 
                                            'PreTHC']

In [ ]:
#further clean dataframe: only keep columns of interest and required metadata
data_RH = data_RH[['TBIID', 'AUDIT1', 'AUDIT2', 'AUDIT3', 'DOPA', 'DA',
       'DOPAC', 'da_dopa_ratio', 'dopac_da_ratio', 'NE', 'DHPG', 'ne_dopa_ratio',
       'dhpg_ne_ratio', 'CSFPROTEIN', 'LPHRate', 'LPBPSys',
       'LPBPDias', 'PTSD_YN']]

In [ ]:
#check length of data tables (AUDITC has more entries)
print('The length of the multi data table is: ', first_visit_data_mult.shape)
print('The length of the single data table is: ', first_visit_data_single.shape)
print('The length of the TBI data table is: ', first_visit_data_TBI.shape)
print('The length of the RH data table is: ', data_RH.shape)
print('The length of the TBI data table is: ', first_visit_data_PET.shape)
print('The length of the drug_history data table is: ', first_visit_data_drug.shape)

In [ ]:
#joing the data sets on the common key 'TBIID' - this should only keep entries that are shared across all data sets
merge_data_first = pd.merge(first_visit_data_mult, first_visit_data_single, how='inner', on='TBIID', suffixes=('_mult', '_single'), validate='one_to_one')
print(merge_data_first.shape)
merge_data_first = pd.merge(merge_data_first, first_visit_data_TBI, how='inner', on='TBIID', suffixes=('_mult', '_TBI'), validate='one_to_one')
print(merge_data_first.shape)
merge_data_first = pd.merge(merge_data_first, first_visit_data_PET, how='inner', on='TBIID', suffixes=('_mult', '_PET'), validate='one_to_one')
#print(merge_data_first.shape)
merge_data_first = pd.merge(merge_data_first, first_visit_data_drug, how='inner', on='TBIID', suffixes=('_mult', '_drug'), validate='one_to_one')
#print(merge_data_first.shape)
merge_data_first = pd.merge(merge_data_first, data_RH, how='inner', on='TBIID', suffixes=('_mult', '_AUDITC'), validate='one_to_one')
print(merge_data_first.shape)
merge_data_first.head(1)

In [ ]:
#further clean dataframe: rearange columns and only keep columns of interest and required metadata    
merge_data_first = merge_data_first[['TBIID', 'Group', 
                                     'VisitSeq_mult', 'VisitSeq_single', 'VisitSeq_TBI', 'VisitSeq_PET',
                                     'MeasureDate_mult', 'MeasureDate_single', 'MeasureDate_TBI', 'MeasureDate_PET',
                                     'GType', 'Race', 'Hispanic', 'Handedness', 'ScreenAge', 'Education', 'PsyEduc', 
                                     'cestotal', 'NSITot', 'TBITot', 'CAPSTotal', 'PCLTot', 'PTSD_YN', 'PSQItot', 'PHQTot', 'BNITotIm',
                                     'BISTot', 'AUDITtot', 'AUDIT1', 'AUDIT2', 'AUDIT3', 
                                     'PreCaff', 'PreETOH', 'PreNic', 'PreSleep', 'PreTHC',
                                     'DOPA', 'DA', 'DOPAC', 'da_dopa_ratio', 'dopac_da_ratio', 
                                     'NE', 'DHPG', 'ne_dopa_ratio', 'dhpg_ne_ratio',
                                     'CSFPROTEIN', 'LPHRate', 'LPBPSys', 'LPBPDias',
                                     'QKOIorA', 'QKOExpMil', 'QKOAllMil', 'QKOLife', 'QBlstExp', 'QBEACRM', 
                                     'MnthSncBlst', 'QEDist_sum', 'QEDist_mean', 'QEDist_min',
                                     'DvpHA', 'DvpHAAct', 'DvpHASlp', 'DvpHAMd', 'DvpHAStr', 'DvpBP', 'DvpBPAct', 'DvpBPslp', 
                                     'DvpBPMd', 'DvpBPStr',
                                     'tbiDizzy', 'tbiBalan', 'tbiCoord', 'tbiHeada', 'tbiNaus', 'tbiVision', 'tbiLight', 
                                     'tbiHear', 'tbiNoise', 'tbiTingl', 'tbiTstsml', 'tbiAppet', 'tbiConc', 'tbiForget', 
                                     'tbiDecis', 'tbiSlow', 'tbiEnergy', 'tbiSleep', 'tbiAnx', 'tbiSad', 'tbiIrrit', 'tbiOverw',
                                     'tbiDisin', 'tbiWithd', 'tbiRing', 'tbiMoods', 'tbiFight', 'tbiSpch', 
                                     'hrslp', 'PSQI1hr', 'PSQI1min', 'PSQI2', 'PSQI3hr', 'PSQI3min', 'PSQI5a', 
                                     'PSQI5b', 'PSQI5c', 'PSQI5d', 'PSQI5e', 'PSQI5f', 'PSQI5g', 'PSQI5h', 'PSQI5i', 'PSQI5j', 
                                     'PSQI6', 'PSQI7', 'PSQI8', 'PSQI9', 'PSQIc1', 'PSQIc2', 'PSQIc3', 'PSQIc4', 'PSQIc5', 
                                     'PSQIc6', 'PSQIc7', 
                                     'PCL1', 'PCL2', 'PCL3', 'PCL4', 'PCL5', 'PCL6', 'PCL7', 'PCL8', 'PCL9',
                                     'PCL10', 'PCL11', 'PCL12', 'PCL13', 'PCL14', 'PCL15', 'PCL16', 'PCL17',  
                                     'PHQ1', 'PHQ2', 'PHQ3', 'PHQ4', 'PHQ5', 'PHQ6', 'PHQ7', 'PHQ8', 'PHQ9', 
                                     'BNI1Im', 'BNI2Im', 'BNI3Im', 'BNI4Im', 'BNI5Im', 'BNI6Im', 'BNI7Im', 'BNI8Im', 'BNI9Im', 
                                     'BNI10Im', 'BNI11',
                                     'BISAtt', 'BISMtr', 'BISNonpl',  
                                     'PETType', 'Frontal_Mid_l', 'Frontal_Mid_r', 'Insula_l', 'Insula_r', 'Cingulum_Ant_l', 
                                     'Cingulum_Ant_r', 'Amygdala_l', 'Amygdala_r', 'CaudateNucl_l', 'CaudateNucl_r', 
                                     'Putamen_l', 'Putamen_r', 'Pallidum_l', 'Pallidum_r', 'Medulla', 'Midbrain', 'Pons']]

print(merge_data_first.shape)
merge_data_first.head(5)

In [ ]:
merge_data_first.to_csv('merge_data_first.csv')

In [ ]:
#-999 and 'None' are missing values so replace with 'Nan'
merge_data_first = merge_data_first.replace({-999.0: np.nan, 'None': np.nan})

In [ ]:
#explore missing data
print(merge_data_first.shape)
print(merge_data_first[merge_data_first['Group'] == 'T'].shape)
print(merge_data_first[merge_data_first['Group'] == 'T'].isna().sum().sort_values(ascending=False))
merge_data_first[merge_data_first['Group'] == 'T'].isna().sum().sort_values(ascending=False).plot(kind = 'hist')
plt.show()

In [ ]:
dep_vars = ['GType', 'Race', 'Hispanic', 'Handedness', 'ScreenAge',
       'Education', 'PsyEduc', 'cestotal', 'NSITot', 'TBITot',
       'CAPSTotal', 'PCLTot', 'PTSD_YN', 'PSQItot', 'PHQTot', 'BNITotIm',
       'BISTot', 'AUDITtot', 'AUDIT1', 'AUDIT2', 'AUDIT3', 'PreCaff', 'PreETOH',
       'PreNic', 'PreSleep', 'PreTHC','DOPA', 'DA', 'DOPAC', 'da_dopa_ratio', 'dopac_da_ratio', 
       'NE', 'DHPG', 'ne_dopa_ratio', 'dhpg_ne_ratio',
    'CSFPROTEIN', 'LPHRate', 'LPBPSys', 'LPBPDias', 'QKOIorA', 'QKOExpMil', 'QKOAllMil', 'QKOLife', 'QBlstExp',
       'QBEACRM', 'MnthSncBlst', 'QEDist_sum', 'QEDist_mean', 'DvpHA',
       'DvpHAAct', 'DvpHASlp', 'DvpHAMd', 'DvpHAStr', 'DvpBP', 'DvpBPAct',
       'DvpBPslp', 'DvpBPMd', 'DvpBPStr', 'tbiDizzy', 'tbiBalan',
       'tbiCoord', 'tbiHeada', 'tbiNaus', 'tbiVision', 'tbiLight',
       'tbiHear', 'tbiNoise', 'tbiTingl', 'tbiTstsml', 'tbiAppet',
       'tbiConc', 'tbiForget', 'tbiDecis', 'tbiSlow', 'tbiEnergy',
       'tbiSleep', 'tbiAnx', 'tbiSad', 'tbiIrrit', 'tbiOverw', 'tbiDisin',
       'tbiWithd', 'tbiRing', 'tbiMoods', 'tbiFight', 'tbiSpch', 'hrslp',
       'PSQI1hr', 'PSQI1min', 'PSQI2', 'PSQI3hr', 'PSQI3min', 'PSQI5a',
       'PSQI5b', 'PSQI5c', 'PSQI5d', 'PSQI5e', 'PSQI5f', 'PSQI5g',
       'PSQI5h', 'PSQI5i', 'PSQI5j', 'PSQI6', 'PSQI7', 'PSQI8', 'PSQI9',
       'PSQIc1', 'PSQIc2', 'PSQIc3', 'PSQIc4', 'PSQIc5', 'PSQIc6',
       'PSQIc7', 'PCL1', 'PCL2', 'PCL3', 'PCL4', 'PCL5', 'PCL6', 'PCL7',
       'PCL8', 'PCL9', 'PCL10', 'PCL11', 'PCL12', 'PCL13', 'PCL14',
       'PCL15', 'PCL16', 'PCL17', 'PHQ1', 'PHQ2', 'PHQ3', 'PHQ4', 'PHQ5',
       'PHQ6', 'PHQ7', 'PHQ8', 'PHQ9', 'BNI1Im', 'BNI2Im', 'BNI3Im',
       'BNI4Im', 'BNI5Im', 'BNI6Im', 'BNI7Im', 'BNI8Im', 'BNI9Im',
       'BNI10Im', 'BNI11', 'BISAtt', 'BISMtr', 'BISNonpl', 'PETType',
       'Frontal_Mid_l', 'Frontal_Mid_r', 'Insula_l', 'Insula_r',
       'Cingulum_Ant_l', 'Cingulum_Ant_r', 'Amygdala_l', 'Amygdala_r',
       'CaudateNucl_l', 'CaudateNucl_r', 'Putamen_l', 'Putamen_r',
       'Pallidum_l', 'Pallidum_r', 'Medulla', 'Midbrain', 'Pons']

In [ ]:
for param in dep_vars:
    print(param)
    plt.figure(figsize=(10,5))
    try:
        merge_data_first.groupby(['Group'])[param].mean().plot(kind='bar', yerr=merge_data_first.groupby(['Group'])[param].sem())
        plt.ylabel([param])
    #plt.savefig(str(param + '.png'))
        plt.show()
    except:
        pass

In [ ]:
corr = merge_data_first.groupby('Group').corr()
#fig, ax = plt.subplots(figsize=(40, 40))
#sns.heatmap(corr, center=0)

In [ ]:
AUDITC_cols = ['AUDITtot',
       'AUDIT1', 'AUDIT2', 'AUDIT3']

In [ ]:
merge_data_first = merge_data_first[merge_data_first["TBIID"] != 'C010']
merge_data_first = merge_data_first[merge_data_first["TBIID"] != 'T080']

In [ ]:
#corr.to_csv('corr.csv')
merge_data_first.to_csv('merge_data_first.csv')

In [ ]:
AUDIT_tot_counts = merge_data_first.groupby('Group')['AUDITtot'].value_counts()
AUDIT_1_counts = merge_data_first.groupby('Group')['AUDIT1'].value_counts()
AUDIT_2_counts = merge_data_first.groupby('Group')['AUDIT2'].value_counts()
AUDIT_3_counts = merge_data_first.groupby('Group')['AUDIT3'].value_counts()
AUDIT_tot_counts.to_csv('AUDIT_tot_counts.csv')
AUDIT_1_counts.to_csv('AUDIT_1_counts.csv')
AUDIT_2_counts.to_csv('AUDIT_2_counts.csv')
AUDIT_3_counts.to_csv('AUDIT_3_counts.csv')

In [ ]:
AUDIT_tot_perc = merge_data_first.groupby('Group')['AUDITtot'].value_counts() / merge_data_first.groupby('Group')['AUDITtot'].count()
AUDIT_1_perc = merge_data_first.groupby('Group')['AUDIT1'].value_counts() / merge_data_first.groupby('Group')['AUDIT1'].count()
AUDIT_2_perc = merge_data_first.groupby('Group')['AUDIT2'].value_counts() / merge_data_first.groupby('Group')['AUDIT2'].count()
AUDIT_3_perc = merge_data_first.groupby('Group')['AUDIT3'].value_counts() / merge_data_first.groupby('Group')['AUDIT3'].count()
AUDIT_tot_perc.to_csv('AUDIT_tot_perc.csv')
AUDIT_1_perc.to_csv('AUDIT_1_perc.csv')
AUDIT_2_perc.to_csv('AUDIT_2_perc.csv')
AUDIT_3_perc.to_csv('AUDIT_3_perc.csv')